In [1]:
import numpy as np
import math
import random
import h5py

import tensorflow as tf
import keras
import keras.backend as K

### Tryout backdoored model

In [2]:
clean_data_filename = "original_data/lab3/data/cl/valid.h5"
poisoned_data_filename = "original_data/lab3/data/bd/bd_valid.h5"
model_filename = "original_data/lab3/models/bd_net.h5"

In [3]:
def data_loader(filepath):
    data = h5py.File(filepath, 'r')
    x_data = np.array(data['data'])
    y_data = np.array(data['label'])
    x_data = x_data.transpose((0,2,3,1))

    return x_data, y_data

In [4]:
cl_x_val, cl_y_val = data_loader(clean_data_filename)
bd_x_val, bd_y_val = data_loader(poisoned_data_filename)

bd_model = keras.models.load_model(model_filename)

cl_label_p = np.argmax(bd_model.predict(cl_x_val), axis=1)
clean_accuracy = np.mean(np.equal(cl_label_p, cl_y_val))*100
print('Clean Classification accuracy:', clean_accuracy)

bd_label_p = np.argmax(bd_model.predict(bd_x_val), axis=1)
asr = np.mean(np.equal(bd_label_p, bd_y_val))*100
print('Attack Success Rate:', asr)

Clean Classification accuracy: 98.64899974019225
Attack Success Rate: 100.0


In [5]:
weights = dict([(layer.name, layer.get_weights()) for layer in bd_model.layers])

### Pruning defense

In [6]:
class Feature_Block(keras.layers.Layer):
    def __init__(self, input_dim=(55, 47, 3)):
        super(Feature_Block, self).__init__()
        # Sub-layers
        self.conv_1 = keras.layers.Conv2D(20, (4, 4), activation='relu', name='conv_1')
        self.pool_1 = keras.layers.MaxPooling2D((2, 2), name='pool_1')
        self.conv_2 = keras.layers.Conv2D(40, (3, 3), activation='relu', name='conv_2')
        self.pool_2 = keras.layers.MaxPooling2D((2, 2), name='pool_2')
        self.conv_3 = keras.layers.Conv2D(60, (3, 3), activation='relu', name='conv_3')
        self.pool_3 = keras.layers.MaxPooling2D((2, 2), name='pool_3')
    
    def call(self, x):
        z = self.pool_1(self.conv_1(x))
        z = self.pool_2(self.conv_2(z))
        z = self.pool_3(self.conv_3(z))
        return z

class Interpretation_Block(keras.layers.Layer):
    def __init__(self, input_dim=(5, 4, 60)):
        super(Interpretation_Block, self).__init__()
        # Sub-layers
        self.flat_1 = keras.layers.Flatten()
        self.fc_1 = keras.layers.Dense(160, name='fc_1')
        self.conv_4 = keras.layers.Conv2D(80, (2, 2), activation='relu', name='conv_4')
        self.flat_2 = keras.layers.Flatten()
        self.fc_2 = keras.layers.Dense(160, name='fc_2')
        self.merge = keras.layers.Add()
        self.add_1 = keras.layers.Activation('relu')
        self.y = keras.layers.Dense(1283, activation='softmax', name='output')
    
    def call(self, x):
        z1 = self.fc_1(self.flat_1(x))
        z2 = self.fc_2(self.flat_2(self.conv_4(x)))
        return self.y(self.add_1(self.merge([z1, z2])))

In [17]:
def Feature_Net():
	x = keras.Input(shape=(55, 47, 3), name='input')
	feat = Feature_Block()(x)
	model = keras.Model(inputs=x, outputs=feat)
	model.set_weights(weights['conv_1'] + weights['conv_2'] + weights['conv_3'])
	return model

def Interpretation_Net():
	x = keras.Input(shape=(5, 4, 60), name='input')
	y = Interpretation_Block()(x)
	model = keras.Model(inputs=x, outputs=y)
	model.set_weights(weights['fc_1'] + weights['conv_4'] + weights['fc_2'] + weights['output'])
	return model

In [18]:
feature_net = Feature_Net()
activations = feature_net.predict(cl_x_val)

In [20]:
interpretation_net = Interpretation_Net()
o = interpretation_net.predict(activations)

In [24]:
def Pruned_Net(custom_mask=None):
    mask = K.variable(custom_mask)
    masked = keras.layers.Lambda(lambda x: x * mask)(feature_net.output)
    model = keras.Model(inputs=feature_net.input, outputs=interpretation_net(masked))
    return model

In [54]:
prune_mask = np.ones((5, 4, 60))

In [55]:
pruned_net = Pruned_Net(prune_mask)

The following Variables were used a Lambda layer's call (lambda_13), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(5, 4, 60) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.


In [56]:
cl_label_p = np.argmax(pruned_net.predict(cl_x_val), axis=1)

In [66]:
class Custom_Layer(keras.layers.Layer):
    def __init__(self):
        super(Custom_Layer, self).__init__()
    
    def call(self, inputs):
        p1, p2 = inputs
        y1, y2 = np.argmax(p1, axis=1), np.argmax(p2, axis=1)
        o1 = np.where(y1 == y2, y1, 0)
        o2 = np.where(y1 == y2, 0, 1)
        return np.concatenate([o1, o2])

class Repaired_Model(keras.Model):
    def __init__(self, custom_mask):
        super(Repaired_Model, self).__init__()
        self.feature = pruned_net.layers[1]
        self.interpretation = interpretation_net.layers[1]
        mask = K.variable(custom_mask)
        self.mask = keras.layers.Lambda(lambda x: x * mask)
    
    def call(self, x):
        z = self.feature(x)
        z1 = self.interpretation(z)
        z2 = self.interpretation(self.mask(z))
        y1, y2 = K.argmax(z1, axis=1), K.argmax(z2, axis=1)
        o1 = tf.where(y1 == y2, z1, 0.)
        o2 = tf.where(y1 == y2, 0., 1.)
        o2 = K.expand_dims(o2, axis=1)
        return tf.concat([o1, o2], axis=1)
    
def Repaired_Net(custom_mask=None):
    return Repaired_Model(custom_mask)

In [67]:
repaired_model = Repaired_Net(prune_mask)
print(len(repaired_model.get_weights()))

14


In [68]:
bd_label_p = np.argmax(repaired_model.predict(bd_x_val), axis=1)

The following Variables were used a Lambda layer's call (lambda_17), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(5, 4, 60) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.


InvalidArgumentError:  condition [32], then [32,1283], and else [] must be broadcastable
	 [[node repaired__model_14/SelectV2
 (defined at C:\Users\xiaot\Anaconda3\envs\torch_keras\lib\site-packages\tensorflow\python\util\tf_stack.py:193)
]] [Op:__inference_predict_function_12155]

Errors may have originated from an input operation.
Input Source operations connected to node repaired__model_14/SelectV2:
In[0] repaired__model_14/Equal:	
In[1] repaired__model_14/interpretation__block_1/output/Softmax:	
In[2] repaired__model_14/SelectV2/e:

Operation defined at: (most recent call last)
>>>   File "C:\Users\xiaot\Anaconda3\envs\torch_keras\lib\runpy.py", line 194, in _run_module_as_main
>>>     return _run_code(code, main_globals, None,
>>> 
>>>   File "C:\Users\xiaot\Anaconda3\envs\torch_keras\lib\runpy.py", line 87, in _run_code
>>>     exec(code, run_globals)
>>> 
>>>   File "C:\Users\xiaot\Anaconda3\envs\torch_keras\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
>>>     

In [69]:
repaired_model.save('try')

AssertionError: Tried to export a function which references 'untracked' resource Tensor("12849:0", shape=(), dtype=resource). TensorFlow objects (e.g. tf.Variable) captured by functions must be 'tracked' by assigning them to an attribute of a tracked object or assigned to an attribute of the main object directly.

 Trackable Python objects referring to this tensor (from gc.get_referrers, limited to two hops):
<tf.Variable 'Variable:0' shape=(5, 4, 60) dtype=float32>